<a href="https://colab.research.google.com/github/Hrithik-12/converailabsassignment/blob/main/assignment%20solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install -q yt-dlp openai-whisper ffmpeg-python

# Download the YouTube audio and convert to wav (16kHz mono)
import subprocess, glob
from yt_dlp import YoutubeDL

YOUTUBE_URL = "https://www.youtube.com/watch?v=4ostqJD3Psc"
OUTPUT_WAV = "call.wav"

# Download audio
ydl_opts = {'format': 'bestaudio/best', 'outtmpl': 'call_audio.%(ext)s'}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([YOUTUBE_URL])

src = glob.glob("call_audio.*")[0]
# Convert to wav (16kHz mono)
subprocess.run(["ffmpeg", "-y", "-i", src, "-ac", "1", "-ar", "16000", OUTPUT_WAV])

print("Saved as:", OUTPUT_WAV)

# Transcribe with Whisper (tiny model for speed)
import whisper
model = whisper.load_model("tiny")
result = model.transcribe(OUTPUT_WAV, language="en")

print("\n--- TRANSCRIPT ---\n")
print(result["text"])
with open("transcript.txt", "w") as f:
    f.write(result["text"])
print("Transcript saved as transcript.txt")


[youtube] Extracting URL: https://www.youtube.com/watch?v=4ostqJD3Psc
[youtube] 4ostqJD3Psc: Downloading webpage
[youtube] 4ostqJD3Psc: Downloading tv simply player API JSON
[youtube] 4ostqJD3Psc: Downloading tv client config
[youtube] 4ostqJD3Psc: Downloading tv player API JSON
[info] 4ostqJD3Psc: Downloading 1 format(s): 251
[download] call_audio.webm has already been downloaded
[download] 100% of    1.99MiB
Saved as: call.wav


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



--- TRANSCRIPT ---

 Thank you for calling me son. My name is Lauren. Can I have your name? Yes, my name is John Smith. Thank you, John. How can I help you? I was just calling about to see how much it would cost to update the map in my car. I'd be happy to help you with that today. Did you receive a mail or from us? I did. Do you need the customer number? Yes, please. Okay. It's 15243. Thank you and the year making model of your vehicle. Yeah, I have a 2009 Nissan Altaman. So nice car. Yeah, thank you. We really enjoy it. Okay. I think I found your profile here. Can I have you verify your address and phone number? Please. Yes. It's 1265 North Research Way. That's an RM UTA-84097. And my phone number is A01-431-1000. Thanks, John. I located your information. The newest version we have available for your vehicle is version 7.7, which was released in March of 2012. The price of the new map is $99. Push it, being attacked. Let me go ahead and set up this order for you. Can we wait just a 

In [ ]:
# transcript_text = """
# Thank you for calling me son. My name is Lauren. Can I have your name?
# Yes, my name is John Smith. Thank you, John. How can I help you?
# I was just calling about to see how much it would cost to update the map in my car.
# I'd be happy to help you with that today. Did you receive a mail or from us?
# I did. Do you need the customer number? Yes, please. Okay. It's 15243.
# Thank you and the year making model of your vehicle. Yeah, I have a 2009 Nissan Altaman.
# So nice car. Yeah, thank you. We really enjoy it. Okay. I think I found your profile here.
# Can I have you verify your address and phone number? Please.
# Yes. It's 1265 North Research Way. That's an RM UTA-84097.
# And my phone number is A01-431-1000. Thanks, John. I located your information.
# The newest version we have available for your vehicle is version 7.7,
# which was released in March of 2012. The price of the new map is $99.
# Push it, being attacked. Let me go ahead and set up this order for you.
# Can we wait just a second? I'm not really sure if I can afford it right now.
# All right. Well, here are a few reasons to consider purchasing today.
# It looks as though you haven't updated your vehicle for three years.
# So that would be the equivalent of getting three years worth of updates for the price of one.
# Okay. In addition, special offers like the current promotion don't come around too often.
# I would definitely recommend taking advantage of the extra $50 off before it expires.
# Yeah, that does sound pretty good. If I set this order up for you now,
# it'll ship out today and for $50 less.
# Do you have your credit card handy and I can place this order for you now?
# Yeah. Let's go ahead and use your visa.
# """

transcript_text="""Can you help me with this assignment
I’m stuck on the regex part!
Do you know how to split text into sentences???
Yes, I can show you.
Cool thanks.
"""

# ===================================================
# APPROACH 1: Regex-based Question Detection (Simple)
# ===================================================

import re

# Define patterns
QUESTION_START_RE = re.compile(
    r'^(who|what|when|where|why|how|is|are|do|does|did|can|could|would|should|will|may|might)\b',
    re.I
)
QUESTION_ANYWHERE_RE = re.compile(
    r'\b(do you|did you|do we|can you|could you|would you|are you|is there|have you|may i|shall we|should we|do they|did they|could I|can I)\b',
    re.I
)

# Split text into sentences (basic split by punctuation)
regex_sentences = re.split(r'(?<=[.?!])\s+', transcript_text)

# Detect questions
regex_questions = []
for s in regex_sentences:
    s_clean = s.strip()
    if not s_clean:
        continue
    if s_clean.endswith("?") or QUESTION_START_RE.match(s_clean) or QUESTION_ANYWHERE_RE.search(s_clean):
        regex_questions.append(s_clean)

print("=== Approach 1: Regex-based ===")
print(f"Total sentences: {len(regex_sentences)}")
print(f"Questions detected: {len(regex_questions)}\n")
for i, q in enumerate(regex_questions, 1):
    print(f"{i}. {q}")

=== Approach 1: Regex-based ===
Total sentences: 5
Questions detected: 2

1. Can you help me with this assignment  
I’m stuck on the regex part!
2. Do you know how to split text into sentences???


In [ ]:
from collections import defaultdict

def calculate_talk_time_ratio(transcript_text):
    """
    Calculates talk-time ratio (%) for a transcript separated by '#' for turns.
    Assumes two speakers alternating.
    """
    # Split by '#' and clean whitespace
    turns = [turn.strip() for turn in transcript_text.split('#') if turn.strip()]

    talk_time = defaultdict(int)
    speakers = ["Speaker 1", "Speaker 2"]

    # Assign alternating speakers
    for i, turn in enumerate(turns):
        speaker = speakers[i % 2]
        talk_time[speaker] += len(turn.split())

    total_words = sum(talk_time.values())
    talk_time_ratio = {speaker: (words / total_words) * 100 for speaker, words in talk_time.items()}

    return talk_time_ratio


# === Example Usage ===
transcript = """
Thank you for calling me son. My name is Lauren. Can I have your name?
# Yes, my name is John Smith. Thank you, John. How can I help you?
# I was just calling about to see how much it would cost to update the map in my car.
# I'd be happy to help you with that today. Did you receive a mail or from us?
# I did. Do you need the customer number? Yes, please. Okay. It's 15243.
# Thank you and the year making model of your vehicle. Yeah, I have a 2009 Nissan Altaman.
# So nice car. Yeah, thank you. We really enjoy it. Okay. I think I found your profile here.
# Can I have you verify your address and phone number? Please.
# Yes. It's 1265 North Research Way. That's an RM UTA-84097.
# And my phone number is A01-431-1000. Thanks, John. I located your information.
# The newest version we have available for your vehicle is version 7.7,
# which was released in March of 2012. The price of the new map is $99.
# Push it, being attacked. Let me go ahead and set up this order for you.
# Can we wait just a second? I'm not really sure if I can afford it right now.
# All right. Well, here are a few reasons to consider purchasing today.
# It looks as though you haven't updated your vehicle for three years.
# So that would be the equivalent of getting three years worth of updates for the price of one.
# Okay. In addition, special offers like the current promotion don't come around too often.
# I would definitely recommend taking advantage of the extra $50 off before it expires.
# Yeah, that does sound pretty good. If I set this order up for you now,
# it'll ship out today and for $50 less.
# Do you have your credit card handy and I can place this order for you now?
# Yeah. Let's go ahead and use your visa.
"""

ratio = calculate_talk_time_ratio(transcript)
for speaker, percent in ratio.items():
    print(f"{speaker}: {percent:.2f}%")


Speaker 1: 50.31%
Speaker 2: 49.69%


In [ ]:
from collections import defaultdict

# Example transcript (replace with your actual transcript_text)
transcript_text = """
Thank you for calling me son. My name is Lauren. Can I have your name?
# Yes, my name is John Smith. Thank you, John. How can I help you?
# I was just calling about to see how much it would cost to update the map in my car.
# I'd be happy to help you with that today. Did you receive a mail or from us?
# I did. Do you need the customer number? Yes, please. Okay. It's 15243.
# Thank you and the year making model of your vehicle. Yeah, I have a 2009 Nissan Altaman.
# So nice car. Yeah, thank you. We really enjoy it. Okay. I think I found your profile here.
# Can I have you verify your address and phone number? Please.
# Yes. It's 1265 North Research Way. That's an RM UTA-84097.
# And my phone number is A01-431-1000. Thanks, John. I located your information.
# The newest version we have available for your vehicle is version 7.7,
# which was released in March of 2012. The price of the new map is $99.
# Push it, being attacked. Let me go ahead and set up this order for you.
# Can we wait just a second? I'm not really sure if I can afford it right now.
# All right. Well, here are a few reasons to consider purchasing today.
# It looks as though you haven't updated your vehicle for three years.
# So that would be the equivalent of getting three years worth of updates for the price of one.
# Okay. In addition, special offers like the current promotion don't come around too often.
# I would definitely recommend taking advantage of the extra $50 off before it expires.
# Yeah, that does sound pretty good. If I set this order up for you now,
# it'll ship out today and for $50 less.
# Do you have your credit card handy and I can place this order for you now?
# Yeah. Let's go ahead and use your visa.
"""

# --- Split transcript into turns ---
turns = [turn.strip() for turn in transcript_text.split('#') if turn.strip()]

# --- Initialize speakers ---
speakers = ["Speaker 1", "Speaker 2"]  # alternating
talk_time = defaultdict(int)
longest_monologue = defaultdict(int)
questions_count = defaultdict(int)

# --- Process each turn ---
for i, turn in enumerate(turns):
    speaker = speakers[i % 2]
    words = len(turn.split())

    # Total words per speaker
    talk_time[speaker] += words

    # Longest monologue (by word count)
    longest_monologue[speaker] = max(longest_monologue.get(speaker, 0), words)

    # Count questions
    questions_count[speaker] += turn.count('?')

# --- Calculate talk-time percentage ---
total_words = sum(talk_time.values())
talk_time_ratio = {s: (w / total_words) * 100 for s, w in talk_time.items()}

# --- Output results ---
print("\n--- TALK-TIME RATIO (%) ---")
for speaker, percent in talk_time_ratio.items():
    print(f"{speaker}: {percent:.2f}%")

print("\n--- LONGEST MONOLOGUE (words) ---")
for speaker, words in longest_monologue.items():
    print(f"{speaker}: {words}")

print("\n--- NUMBER OF QUESTIONS ---")
for speaker, q_count in questions_count.items():
    print(f"{speaker}: {q_count}")



--- TALK-TIME RATIO (%) ---
Speaker 1: 50.31%
Speaker 2: 49.69%

--- LONGEST MONOLOGUE (words) ---
Speaker 1: 19
Speaker 2: 17

--- NUMBER OF QUESTIONS ---
Speaker 1: 2
Speaker 2: 5


In [ ]:
import google.generativeai as genai
from google.colab import userdata

# Get API key securely from Colab Secrets
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"Error: Could not retrieve API key. Make sure to set a secret named 'GOOGLE_API_KEY' in Colab.")

# Initialize the Gemini model
model = genai.GenerativeModel('gemini-2.5-flash')

# The full transcript text
# transcript_text = """
# Thank you for calling me son. My name is Lauren. Can I have your name? Yes, my name is John Smith. Thank you, John. How can I help you? I was just calling about to see how much it would cost to update the map in my car. I'd be happy to help you with that today. Did you receive a mail or from us? I did. Do you need the customer number? Yes, please. Okay. It's 15243. Thank you and the year making model of your vehicle. Yeah, I have a 2009 Nissan Altaman. So nice car. Yeah, thank you. We really enjoy it. Okay. I think I found your profile here. Can I have you verify your address and phone number? Please. Yes. It's 1265 North Research Way. That's an RM UTA-84097. And my phone number is A01-431-1000. Thanks, John. I located your information. The newest version we have available for your vehicle is version 7.7, which was released in March of 2012. The price of the new map is $99. Push it, being attacked. Let me go ahead and set up this order for you. Can we wait just a second? I'm not really sure if I can afford it right now. All right. Well, here are a few reasons to consider purchasing today. It looks as though you haven't updated your vehicle for three years. So that would be the equivalent of getting three years worth of updates for the price of one. Okay. In addition, special offers like the current promotion don't come around too often. I would definitely recommend taking advantage of the extra $50 off before it expires. Yeah, that does sound pretty good. If I set this order up for you now, it'll ship out today and for $50 less. Do you have your credit card handy and I can place this order for you now? Yeah. Let's go ahead and use your visa.
# """

transcript_text = """
Thank you for calling me son. My name is Lauren. Can I have your name?
# Yes, my name is John Smith. Thank you, John. How can I help you?
# I was just calling about to see how much it would cost to update the map in my car.
# I'd be happy to help you with that today. Did you receive a mail or from us?
# I did. Do you need the customer number? Yes, please. Okay. It's 15243.
# Thank you and the year making model of your vehicle. Yeah, I have a 2009 Nissan Altaman.
# So nice car. Yeah, thank you. We really enjoy it. Okay. I think I found your profile here.
# Can I have you verify your address and phone number? Please.
# Yes. It's 1265 North Research Way. That's an RM UTA-84097.
# And my phone number is A01-431-1000. Thanks, John. I located your information.
# The newest version we have available for your vehicle is version 7.7,
# which was released in March of 2012. The price of the new map is $99.
# Push it, being attacked. Let me go ahead and set up this order for you.
# Can we wait just a second? I'm not really sure if I can afford it right now.
# All right. Well, here are a few reasons to consider purchasing today.
# It looks as though you haven't updated your vehicle for three years.
# So that would be the equivalent of getting three years worth of updates for the price of one.
# Okay. In addition, special offers like the current promotion don't come around too often.
# I would definitely recommend taking advantage of the extra $50 off before it expires.
# Yeah, that does sound pretty good. If I set this order up for you now,
# it'll ship out today and for $50 less.
# Do you have your credit card handy and I can place this order for you now?
# Yeah. Let's go ahead and use your visa.
"""

# The prompt with specific instructions for Gemini
prompt = f"""
Analyze the following call transcript. Your analysis should provide two distinct sections:

1.  **Sentiment Summary**: Provide an overall sentiment score for the call on a scale of 1-10 (1 being very negative, 10 being very positive). provide that either is positive or negative .

2.  **Actionable Insights**: Based on the conversation, provide only 1 actionable recommendations for the representative to improve their performance on future calls. The insights should be practical and tied directly to the conversation's content.

3. also give a single answer one word to return which person  is sales agent and customer

Here is the transcript to analyze:
---
{transcript_text}
---
"""

# Send the prompt to the model and print the result
response = model.generate_content(prompt)
print(response.text)

Here's an analysis of the call transcript:

### 1. Sentiment Summary

*   **Overall Sentiment Score**: 8/10
*   **Sentiment**: Positive

### 2. Actionable Insights

1.  **Proactively Present Value with Price**: Instead of waiting for the customer's hesitation, the representative should proactively integrate key benefits and promotional offers when first stating the price. For example, "The price of the new map is $99, but with our current promotion, you'll get $50 off today, bringing you up-to-date with three years of road changes." This approach can preempt objections and immediately establish value.

### 3. Sales Agent and Customer

*   **Sales Agent**: Lauren
*   **Customer**: John
